# Week 8: Sudoku Solver

<font color='blue'><b>Goals of this notebook:</b></font> In this exercise, we implement a Sudoku solver using an integer program and Python-MIP. As an extra result, we will also see how such a solver can be used to test uniqueness of a solution to a given Sudoku puzzle.

## Solving a Sudoku puzzle: modeling with integer variables

As you most likely all know, a Sudoku is puzzle where the goal is to fill in the cells of a $9 \times 9$ grid with integers from $1$ to $9$ such that
- there is precisely one number per cell,
- no row contains two equal numbers,
- no column contains two equal numbers, and
- no one of the nine 3x3 squares that the grid can be partitioned in contains two equal numbers.

An example of a Sudoku puzzle is given below.

<div style="background-color:white">
<center>
    <img src="08_sudoku_example.png", style="padding-top: 10px;">
</center>
</div>

To solve such a Sudoku puzzle, we have to decide which numbers to assign to which cell. These decisions can be modelled by integral decision variables $x_{ijk}$ for $i,j,k\in\{1,\ldots,9\}$ such that

$$
x_{ijk} = \begin{cases} 1 & \text{if cell $(i,j)$ of the Sudoku contains number $k$}\\ 0 & \text{else} \end{cases}\enspace. 
$$

The question is how to set up suitable constraints that guarantee that a feasible $\{0,1\}$-point $x$ does indeed correspond to a solution of the Sudoku.


<b>Your first task:</b> Come up with linear constraints in the variables $x_{ijk}$ that model the conditions imposed on a valid Sudoku solution, i.e., make sure that any $\{0,1\}$-solution of your system corresponds to a feasible solution of a given Sudoku.

*Write your answer here.*

## Implementing integer programs in Python-MIP

We will implement the integer program in Python using the package Python-MIP. If you have not yet installed the package, you can install it using the command below or download it from its official website: https://www.python-mip.com/.

In [1]:
!pip install mip


Implementing integer programs in `mip` is almost the same as implementing linear programs using `PuLP` - except that you'll have to declare that you want to put integrality conditions on your variables. We use the simple IP below for demonstration:

$$\begin{equation}
\begin{array}{lccl}
    \min          &x  \\
    \text{s.t. }  &x &\geq &4.5\\
                  &x &\in  &\mathbb{z}
\end{array}
\end{equation}$$

In [2]:
import mip

simpleProblem = mip.Model(name="Simple_IP_example", sense=mip.MINIMIZE)

x = simpleProblem.add_var(name="x", var_type=mip.INTEGER)

simpleProblem.objective = x

simpleProblem += x >= 4.5

simpleProblem.optimize()


<OptimizationStatus.OPTIMAL: 0>

In [3]:
print(x.x)


5.0


Note that the `add_var` takes an additional (optional) argument `var_type`, which we can be set to `INTEGER`, `BINARY` or `CONTINUOUS` (the latter one being the default). For the Sudoku problem above, you might want to use binary variables. You can visit the [quick start page](https://docs.python-mip.com/en/latest/quickstart.html) of Python-MIP for more details and examples.

<b>Your second task:</b> Implement the constraints that you came up with in the first task in an integer program, and use it to find a solution to a Sudoku problem. Observe that in this problem, we only care about if there is a feasible solution, and all feasible solutions have "equal status" - one feasible solution will not be better than any other ones. Therefore, no maximization or minimization is involved, and thus you can use an IP with a constant objective.

To this end, you can assume that the Sudoku is given to you as a list of $81$ values, each representing a cell of the Sudoku read row by row from left to right; where a $0$ indicates an empty cell. An example is given below. Note that there also is a function to display a Sudoku that is given in the above form.

Make sure that your function returns the Sudoku in the same format as the input.

In [4]:
# Example Sudoku input and Sudoku printing

sudoku1 = [4, 0, 7, 0, 0, 0, 0, 0, 0, 
           0, 3, 5, 0, 9, 7, 4, 0, 0, 
           0, 9, 0, 0, 0, 0, 0, 0, 6, 
           0, 0, 0, 3, 0, 2, 0, 0, 0, 
           6, 0, 0, 0, 8, 0, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 5, 0, 0, 
           0, 0, 0, 4, 0, 0, 0, 1, 8, 
           0, 0, 3, 0, 2, 8, 0, 0, 4, 
           5, 0, 4, 0, 0, 0, 0, 9, 7]

def printSudoku(sudoku):
    # compact Sudoku printing function
    # taken from https://codegolf.stackexchange.com/questions/126930/
    #    draw-a-sudoku-board-using-line-drawing-characters
    q = lambda x,y:x+y+x+y+x
    r = lambda a,b,c,d,e:a+q(q(b*3,c),d)+e+"\n"
    print(((r(*"╔═╤╦╗") + q(q("║ %d │ %d │ %d "*3 + "║\n",r(*"╟─┼╫╢")), r(*"╠═╪╬╣")) +
            r(*"╚═╧╩╝")) % tuple(sudoku)).replace(*"0 "))

printSudoku(sudoku1)


╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │   │ 7 ║   │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 3 │ 5 ║   │ 9 │ 7 ║ 4 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 9 │   ║   │   │   ║   │   │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 3 │   │ 2 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │   │   ║   │ 8 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║ 5 │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 4 │   │   ║   │ 1 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 2 │ 8 ║   │   │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │ 4 ║   │   │   ║   │ 9 │ 7 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [5]:
# Implementation of a Sudoku solver

def sudokuSolver(inputSudoku):
    
    ## Your code goes here.
    
    return outputSudoku


## Checking for uniqueness of the Sudoku solutions

Sudokus are generally agreed to be "real" Sudokus only if they have a unique Solution.

<b>Your third task:</b> Implement a function that checks whether a Sudoko has no solution, a unique solution, or more than one solution. You can reuse the code that you created above for solving the Sudoku. The function should return a tuple `(n, sol)`, where $n\in\{0, 1, 2\}$ depending on whether the Sudoku has zero, one, or at least two solutions, respectively, and `sol` is a list of zero, one, or two solutions of the Sudoku.

If you want a hint, run the following code cell. Do not run it if you want to think about the problem yourself! :)

In [ ]:
## Running this cell will display a hint!

encoded = [79, 98, 115, 101, 114, 118, 101, 32, 116, 104, 97, 116, 32, 115, 111, 108, 118, 105, 110, 103, 32, 97, 32, 83, 117, 100, 111, 107, 117, 32, 100, 105, 100, 32, 110, 111, 116, 32, 117, 115, 101, 32, 116, 104, 101, 32, 111, 98, 106, 101, 99, 116, 105, 118, 101, 32, 102, 117, 110, 99, 116, 105, 111, 110, 32, 111, 102, 32, 116, 104, 101, 32, 73, 80, 46, 32, 79, 110, 99, 101, 32, 121, 111, 117, 32, 102, 111, 117, 110, 100, 32, 111, 110, 101, 32, 115, 111, 108, 117, 116, 105, 111, 110, 44, 32, 99, 97, 110, 32, 121, 111, 117, 32, 101, 120, 112, 108, 111, 105, 116, 32, 116, 104, 101, 32, 102, 97, 99, 116, 32, 116, 104, 97, 116, 32, 121, 111, 117, 32, 99, 97, 110, 32, 99, 104, 111, 111, 115, 101, 32, 116, 104, 101, 32, 111, 98, 106, 101, 99, 116, 105, 118, 101, 32, 116, 111, 32, 115, 101, 101, 32, 105, 102, 32, 121, 111, 117, 32, 99, 97, 110, 32, 102, 105, 110, 100, 32, 97, 110, 111, 116, 104, 101, 114, 32, 115, 111, 108, 117, 116, 105, 111, 110, 63]
print('Hint: ' + ''.join([chr(x) for x in encoded]))


In [7]:
def numberOfSolutions(inputSudoku):
    
    ## Your code goes here.
    
    return (n, sol)



## Testing your code

Among the following three Sudokus, there is one from each category that your function `numberOfSolutions()` should be able to distinguish: One has no solution, one has a unique Solution, and one has two Solutions. Test your implementation on these Sudokus!

In [8]:
sudoku2 = [2, 0, 0, 0, 0, 0, 0, 4, 0, 
           1, 0, 0, 0, 0, 0, 0, 0, 7,
           8, 0, 6, 3, 0, 0, 0, 0, 0,
           0, 5, 0, 0, 0, 7, 3, 0, 1, 
           0, 0, 3, 0, 1, 0, 0, 0, 0, 
           0, 0, 2, 0, 0, 3, 7, 5, 4, 
           0, 0, 7, 0, 0, 5, 0, 0, 0, 
           5, 0, 0, 0, 4, 0, 0, 0, 0, 
           0, 0, 0, 1, 7, 0, 0, 0, 8]
printSudoku(sudoku2)


╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 2 │   │   ║   │   │   ║   │ 4 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║   │   │   ║   │   │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │   │ 6 ║ 3 │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │ 5 │   ║   │   │ 7 ║ 3 │   │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 1 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 2 ║   │   │ 3 ║ 7 │ 5 │ 4 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │ 7 ║   │   │ 5 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │   ║   │ 4 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 1 │ 7 │   ║   │   │ 8 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [9]:
sudoku3 = [0, 0, 0, 6, 0, 7, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 0, 9, 8,
           3, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 2, 0, 6, 0, 0, 
           0, 0, 0, 0, 0, 0, 7, 0, 0, 
           0, 4, 0, 0, 8, 0, 0, 0, 0, 
           1, 0, 0, 0, 0, 0, 0, 2, 3, 
           0, 0, 8, 9, 0, 0, 0, 0, 0, 
           0, 0, 0, 4, 0, 0, 1, 0, 0]
printSudoku(sudoku3)


╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │   │   ║ 6 │   │ 7 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║   │ 9 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │   ║   │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 2 │   ║ 6 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║ 7 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 4 │   ║   │ 8 │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 1 │   │   ║   │   │   ║   │ 2 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 8 ║ 9 │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 4 │   │   ║ 1 │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [10]:
sudoku4 = [0, 6, 0, 0, 0, 0, 0, 7, 4,
           1, 0, 0, 6, 0, 7, 0, 0, 3, 
           7, 0, 0, 0, 0, 0, 0, 0, 0, 
           0, 0, 0, 0, 1, 0, 0, 0, 2, 
           0, 0, 1, 5, 0, 0, 9, 0, 0, 
           9, 0, 0, 8, 0, 0, 0, 1, 0, 
           0, 0, 0, 0, 0, 0, 0, 3, 0, 
           3, 0, 0, 0, 0, 2, 8, 5, 0, 
           0, 0, 9, 0, 0, 4, 0, 0, 0]
printSudoku(sudoku4)


╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │ 6 │   ║   │   │   ║   │ 7 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║ 6 │   │ 7 ║   │   │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │   │   ║   │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 1 │   ║   │   │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 1 ║ 5 │   │   ║ 9 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │   │   ║ 8 │   │   ║   │ 1 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │   │   ║   │ 3 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │   ║   │   │ 2 ║ 8 │ 5 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 9 ║   │   │ 4 ║   │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [11]:
# Test your functions here!

